# [Manage your Money with Python](https://towardsdatascience.com/manage-your-money-with-python-707579202203)

## A Step-by-Step Guide to put you in control of your finances


---

### Step 1: Import Python Libraries


In [ ]:
# !pip install jupyter_dash

In [4]:
import pandas as pd                       #to perform data manipulation and analysis
import numpy as np                        #to cleanse data
from datetime import datetime             #to manipulate dates
import plotly.express as px               #to create interactive charts
import plotly.graph_objects as go         #to create interactive charts
from jupyter_dash import JupyterDash      #to build Dash apps from Jupyter environments
from dash import dcc       #to get components for interactive user interfaces
from dash import html       #to compose the dash layout using Python structures

### Step 2: Get the Data

Then we need to get all our transactions data. This can be done by downloading your transaction data from your online banking into a csv file (most of the banks allow you to do this from their app).

In [22]:
df = pd.read_csv('bk_download.csv') 
df.head()

,date,description,category,amount
0,7/25/2022,USAA FUNDS TRANSFER CR,Uncategorized,200.0
1,7/11/2022,USAA FUNDS TRANSFER DB,Uncategorized,-15.0
2,7/2/2022,USAA FUNDS TRANSFER DB,Uncategorized,-180.0
3,6/30/2022,USAA FUNDS TRANSFER DB,Uncategorized,-1000.0
4,6/29/2022,USAA FUNDS TRANSFER CR,Uncategorized,4500.0


### Step 3: Manipulate the Data and Customize your transactions’ categories
By following this step you can edit your data and personalize the way you want to manage your transactions.

#### Step 3.1: Remove rows containing specific strings

With this simple line you can remove all the rows containing a specific string in a certain column (you can then repeat the command as many times you want to remove other rows). In the example below I remove all the transactions that I do to move money from one bank account to another, which my app labels as Internal.

### Step 3.2: Define all your transactions‘ categories

- Now it’s time to define the buckets which all the transactions should fall into. In my case I chose to use 14 categories:
  - Income
  - Entertainment
  - Food
  - Rent&Bills
  - Travel
  - Transport
  - Sport
  - Transfer
  - Materialistic Desire
  - Cash Withdrawal
  - Gifts
  - Groceries
  - Personal care
  - Other

### Step 3.3: Assign transactions to the correct category

After you have defined all the transactions’ categories you want to use to manage your money, we can start identifying the transactions that are wrongly labelled in the DataFrame and assigning them to the correct category. This is key to shape your final report in the way that best fits your needs.

One way of doing it is again by locating the rows containing a specific string and assign those to a certain category (e.g. all the rows containing Uber, Zipcar or Bird should go under Transport category).

In [23]:
df.head()

,date,description,category,amount
0,7/25/2022,USAA FUNDS TRANSFER CR,Uncategorized,200.0
1,7/11/2022,USAA FUNDS TRANSFER DB,Uncategorized,-15.0
2,7/2/2022,USAA FUNDS TRANSFER DB,Uncategorized,-180.0
3,6/30/2022,USAA FUNDS TRANSFER DB,Uncategorized,-1000.0
4,6/29/2022,USAA FUNDS TRANSFER CR,Uncategorized,4500.0


One way of doing it is again by locating the rows containing a specific string and assign those to a certain category (e.g. all the rows containing Uber, Zipcar or Bird should go under Transport category).

In [24]:
df['category'] = np.where(df['description'].str.contains('Uber|Zipcar|bird|Tfl Travel Charge|Ewa'), 'Transport', df['category'] )


In [27]:
df['category'] = np.where(df['description'].str.contains('Amazon|AMAZON|Amz'),'Materialistic Desire', df['category'])

df['category'] = np.where(df['description'].str.contains('Gym|GYM|Thenx|Footy|Sporting'),'Sport', df['category'] )

df['category'] = np.where(df['description'].str.contains('George MacLean'),'Rent&Bills', df['category'] )

df['category'] = np.where(df['description'].str.contains('UBER|Uber|uber|ZIPCAR|Zipcar|bird|Lim|TFL TRAVEL|Tfl Travel Charge|Ewa'), 'Transport', df['category'] )

df['category'] = np.where(df['description'].str.contains('Wooden|Hackney Cycles'),'Materialistic Desire', df['category'] )

df['category'] = np.where(df['description'].str.contains('itunes|apple|ODEON|Odeon'),'Entertainment', df['category'] )

df['category'] = np.where(df['description'].str.contains('Airbnb|Ryanair|Trainline|trainline|Booking|Flixbus'),'Travel', df['category'] )

Additionally, with the following command, you can also reassign transactions starting with a specific string to a particular category. For example, I’m moving all the transactions with the description starting with “To “ to the Transfer category.

In [25]:
df['starts'] = list( 
    map(lambda x: x.startswith('To '), df['description'])) 
df.loc[df.starts == True, 'category'] = "Transfer"
df.drop('starts', axis=1, inplace=True)

### Step 3.4: Change the categories’ names

To change the existing categories with the ones that we have defined in the previous step we can use the following line of code. As mentioned, an example would be to assign all the transactions related to eating/drinking to a single Food category.

In [28]:
df.category.replace(["Bills", "Expenses", "General", "Housing", "Leisure"], "Other",inplace=True)
df.category.replace(["Coffee", "Eating out", "Takeaway", "Lunch"], "Food",inplace=True)
df.category.replace("Education", "Personal Improvement",inplace=True)
df.category.replace("Drinks", "Entertainment",inplace=True)
df.category.replace("Petrol", "Transport",inplace=True)
df.category.replace("Shopping", "Materialistic Desire",inplace=True)
df.category.replace("Transfers", "Transfer",inplace=True)
df.category.replace("Cash", "Cash Withdrawal",inplace=True)
df.category.replace("Personal Improvement", "Self Improvement",inplace=True)

### Step 4: Design your Monthly Report

After completion of Step 3, we should now have a clean DataFrame that has transactions aggregated by the categories that make more sense to us. We are now ready to design our monthly report, which will consist of 3 different charts.

#### Step 4.1: Net Worth Over Time Chart

The first chart that we are going to create will show how our Net Worth (Income minus Expenses) changes over time.

Let’s add a column to get the year and month of the transaction (since we want to create a monthly report). Then, to create the first graph we simply need to group the DataFrame by the year_month column that we have just created and add a column with the cumulative sum of the amounts.

By using Plotly Go Library we can create a line graph with Net Worth on the Y-axis and Date on the X-axis.

In [29]:
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.strftime('%Y-%m')

Export cleansed csv file


In [32]:
filename = datetime.now().strftime("%Y-%m-%d-%H-%M.csv")

path = "C:/Users/jared.godar/Desktop/misc/NewTransactionFile " +   filename 
df.to_csv (path)

### Net Worth Over Time Chart


In [34]:
Net_Worth_Table = df.groupby('year_month')['amount'].sum().reset_index(name ='sum')
Net_Worth_Table['cumulative sum'] = Net_Worth_Table['sum'].cumsum()
Net_Worth_Table

,year_month,sum,cumulative sum
0,2022-01,-597.40,-597.40
1,2022-02,2121.29,1523.89
2,2022-03,-858.17,665.72
3,2022-04,516.88,1182.60
4,2022-05,-614.21,568.39
5,2022-06,10232.12,10800.51
6,2022-07,-5679.82,5120.69


In [35]:
Net_Worth_Chart = go.Figure(
    data = go.Scatter(x = Net_Worth_Table["year_month"], y = Net_Worth_Table["cumulative sum"]),
    layout = go.Layout(
        title = go.layout.Title(text="Net Worth Over Time")
    )
)
Net_Worth_Chart.update_layout(
    xaxis_title = "Date",
    yaxis_title = "Net Worth (£)",
    hovermode = 'x unified'
    )
Net_Worth_Chart.update_xaxes(
    tickangle = 45)
Net_Worth_Chart.show()

### Total Monthly Expenses Chart


In [36]:
df = df[df.category != "Income"] 
df.amount = df.amount*(-1) 

In [37]:
Total_Monthly_Expenses_Table = df.groupby('year_month')['amount'].sum().reset_index(name='sum')
Total_Monthly_Expenses_Table = Total_Monthly_Expenses_Table.rename(columns={'year_month': 'DATE', 'sum': 'TOTAL EXPENSE'})
Total_Monthly_Expenses_Table

,DATE,TOTAL EXPENSE
0,2022-01,597.40
1,2022-02,-2121.29
2,2022-03,858.17
3,2022-04,-516.88
4,2022-05,614.21
5,2022-06,-10232.12
6,2022-07,5679.82


In [38]:
Total_Monthly_Expenses_Chart = px.bar(Total_Monthly_Expenses_Table, x="DATE", y="TOTAL EXPENSE",title="Total Monthly Expenses")
Total_Monthly_Expenses_Chart.update_yaxes(title='Expenses (£)', visible=True, showticklabels=True)
Total_Monthly_Expenses_Chart.update_xaxes(title='Date',visible=True, showticklabels=True)

Total_Monthly_Expenses_Chart.show()

### Expenses Breakdown Chart


In [39]:
Expenses_Breakdown_Table = pd.pivot_table(df, values = ['amount'], index = ['category', 'year_month'], aggfunc=sum).reset_index()
Expenses_Breakdown_Table.columns = [x.upper() for x in Expenses_Breakdown_Table.columns]
Expenses_Breakdown_Table = Expenses_Breakdown_Table.rename(columns={'YEAR_MONTH': 'DATE'})
Expenses_Breakdown_Table = Expenses_Breakdown_Table[['DATE', 'CATEGORY', 'AMOUNT']]

In [40]:
#Creation of a df with all dates within the range we have data for each category /
#(e.g. for cash withdrawal there are no transactions between MArch 2020 and August 2020 /
#so there is not data point in the graph for those dates and I have to fix that)

Expenses_Breakdown_Table_All_Dates = Expenses_Breakdown_Table.set_index(
    ['DATE', 'CATEGORY']
).unstack(
    fill_value=0
).asfreq(
    'M', fill_value=0
).stack().sort_index(level=1).reset_index()

Expenses_Breakdown_Table_All_Dates['DATE'] = pd.to_datetime(Expenses_Breakdown_Table_All_Dates['DATE'])
Expenses_Breakdown_Table_All_Dates['DATE'] = Expenses_Breakdown_Table_All_Dates['DATE'].dt.strftime('%Y-%m')

In [41]:
#Appending all dates to the original df

Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table.append(Expenses_Breakdown_Table_All_Dates, ignore_index=True)
Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table_Final.drop_duplicates(subset = ['DATE', 'CATEGORY'], keep = 'first')
Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table_Final.sort_values(['DATE', 'CATEGORY'], ascending=[True, False],ignore_index=True)

In [42]:
#creating a df only for the latest date
#I need it to add 0s in case in the latest date there was no transaction for a specific category
#otherwise again the data point for the latest date will not appear in the chart
#(before I fixed only the dates within the date range of the transactions, not the latest date)

latest_date = Expenses_Breakdown_Table_Final['DATE'].max()
df_latest_date = Expenses_Breakdown_Table_Final.loc[Expenses_Breakdown_Table_Final['DATE'] == latest_date]

In [43]:
categories_lst = ['Travel','Transport','Transfer','Other','Materialistic Desire','Groceries','Gifts','Food','Entertainment','Cash Withdrawal','Rent&Bills','Personal Care','Sport']
missing_cat_latest_date = pd.DataFrame({'CATEGORY': list(set(df_latest_date['CATEGORY']) ^ set(categories_lst))})
missing_cat_latest_date['AMOUNT']=0.0
missing_cat_latest_date['DATE']= df_latest_date['DATE'].max()
missing_cat_latest_date = missing_cat_latest_date [['DATE', 'CATEGORY', 'AMOUNT']]
missing_cat_latest_date

,DATE,CATEGORY,AMOUNT
0,2022-07,Rent&Bills,0.0
1,2022-07,Materialistic Desire,0.0
2,2022-07,Utilities,0.0
3,2022-07,Healthcare/Medical,0.0
4,2022-07,Hobbies,0.0
...,...,...,...
23,2022-07,Clothing/Shoes,0.0
24,2022-07,Online Services,0.0
25,2022-07,Credit Card Payments,0.0
26,2022-07,Other Expenses,0.0


In [44]:
#Appending the categories with no transactions for the latest date to the final df for this chart

Expenses_Breakdown_Table_Final = Expenses_Breakdown_Table_Final.append (missing_cat_latest_date)

In [45]:
Expenses_Breakdown_Chart = px.line(Expenses_Breakdown_Table_Final, x='DATE', y="AMOUNT", title="Expenses Breakdown", color = 'CATEGORY')
Expenses_Breakdown_Chart.update_yaxes(title='Expenses (£)', visible=True, showticklabels=True)
Expenses_Breakdown_Chart.update_xaxes(title='Date', visible=True, showticklabels=True)

Expenses_Breakdown_Chart.show()

In [46]:
# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    
    html.Div([
        html.H1(str(latest_date)+" Personal Finance Summary",style={'text-align':'center'}),
        dcc.Graph(figure = Net_Worth_Chart)
    ]),
  
    html.Div([
        dcc.Graph(figure = Total_Monthly_Expenses_Chart)
    ]),
    
    html.Div([
        dcc.Graph(figure = Expenses_Breakdown_Chart)

    ])
])
    
# Run app and display result
app.run_server(mode='external')

#CLick on the link below to access the "Personal Finances Summary"

Dash app running on http://127.0.0.1:8050/
